In [ ]:
!pip install datasets transformers[sentencepiece]==4.28.0
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 7.4 MB/s eta 0:00:00


# Libraries & Functions

In [ ]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
import torch
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from tokenizers import BertWordPieceTokenizer
from transformers import AutoTokenizer, DataCollatorWithPadding, BertTokenizerFast, DistilBertTokenizerFast,  TrainingArguments, Trainer, AutoModelForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_predict

In [ ]:
!git config --global user.email "viktor.domazetoski@hotmail.com"
!git config --global user.name "ViktorDo1"

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_names = ["BERT", "BioBERT"]
checkpoint_names = ["bert-base-cased", "dmis-lab/biobert-base-cased-v1.2"]

In [ ]:
def prepare_data(X, y):
  data = []
  for i, (sequence, label) in enumerate(zip(X, y)):
    data.append( 
        {"text":sequence, 
        "label":label,
        "idx":i
        })
  df = pd.DataFrame(data)
  return Dataset.from_pandas(df)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length = 512)

In [ ]:
from sklearn import metrics

def calculate_scores(y_test, y_pred, average = "binary"):
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = average)
    recall = recall_score(y_test, y_pred, average = average)
    f1 = f1_score(y_test, y_pred, average = average)
    
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    auc = metrics.auc(fpr, tpr)

    return [accuracy, precision, recall, f1, auc]

# Input Data

In [ ]:
raw_datasets = dict()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
colab_dir = "drive/MyDrive/Text_BioMacro/"

In [ ]:
!ls "drive/MyDrive/Text_BioMacro/"

prepared_lpi_negatives.csv  prepared_predicts_negatives.csv
prepared_lpi_positives.csv  prepared_predicts_positives.csv


## Living Planet Index

In [ ]:
dataset_name = "LPI"
columns = ["Abstract", "Journal", "Title"]
LPI_negatives = pd.read_csv(colab_dir + "prepared_lpi_negatives.csv", usecols=columns)
LPI_negatives["label"] = 0

LPI_positives = pd.read_csv(colab_dir + "prepared_lpi_positives.csv", usecols=columns)
LPI_positives["label"] = 1


raw_datasets[dataset_name] = pd.concat([LPI_positives, LPI_negatives])

del LPI_positives
del LPI_negatives

In [ ]:
raw_datasets[dataset_name]

,Abstract,Journal,Title,label
0,even though intensive aquaculture production o...,Freshwater Biology,aquaculture non native salmonid invasions and ...,1
1,because sea otters enhydra lutris exert a wide...,Ecology,bald eagles and sea otters in the aleutian arc...,1
2,interactions between sea otters enhydra lutris...,Marine Ecology Progress Series,changes in sea urchins and kelp following a re...,1
3,bacterial abundance production and extracellul...,Marine Biology,microbial activity and carbon nitrogen and pho...,1
4,the main objective of many conservation progra...,Ecological Applications,density dependent productivity depression in p...,1
...,...,...,...,...
4995,for any enzyme catalyzed reaction to occur the...,The ISME journal,relationships between protein encoding gene ab...,0
4996,high intensity functional training hift is a p...,Military medicine,is high intensity functional training hift cro...,0
4997,the developmental plasticity of plants relies ...,Proceedings of the National Academy of Science...,differential tor activation and cell prolifera...,0
4998,ocean acidification produced by dissolution of...,Proceedings of the National Academy of Science...,global declines in oceanic nitrification rates...,0


## PREDICTS

In [ ]:
dataset_name = "PREDICTS"
columns = ["Abstract", "Journal", "Title"]
PREDICTS_negatives = pd.read_csv(colab_dir + "prepared_predicts_negatives.csv", usecols=columns)
PREDICTS_negatives["label"] = 0

PREDICTS_positives = pd.read_csv(colab_dir + "prepared_predicts_positives.csv", usecols=columns)
PREDICTS_positives["label"] = 1

raw_datasets[dataset_name] = pd.concat([PREDICTS_positives, PREDICTS_negatives])

del PREDICTS_positives
del PREDICTS_negatives

In [ ]:
raw_datasets[dataset_name]

,Abstract,Journal,Title,label
0,bees are believed to be dominant pollen vector...,Journal of Applied Ecology,bee diversity along a disturbance gradient in ...,1
1,the maintenance of grasslands as distinct habi...,Conservation Biology,grazing intensity and the diversity of grassho...,1
2,male euglossine bees were sampled with chemica...,Biotropica,abundance and diversity of euglossine bees in ...,1
3,niche breadth of species has been hypothesized...,The American Naturalist,ecological specialization and susceptibility t...,1
4,bumblebees hymenoptera apidae are important po...,Oikos,use of genetic markers to quantify bumblebee f...,1
...,...,...,...,...
4995,we tested the hypothesis that the appearance o...,The Science of the total environment,exo enzymatic activities and dissolved organic...,0
4996,given a constantly increasing urban population...,The Science of the total environment,numerical study of the impact of vegetation co...,0
4997,musty and earthy odors frequently characterize...,Water research,contribution of streptomyces in sediment to ea...,0
4998,we selected five typical tree species includin...,Tree physiology,utilization of lightflecks by seedlings of fiv...,0


## Preprocess Data

In [ ]:
preprocessed_datasets_dict = {}

In [ ]:
for dataset_name in list(raw_datasets.keys()):
  for text_data in ["Title", "Abstract"]:
    X_train, X_test, \
    y_train, y_test, \
    indices_train, indices_test \
    = train_test_split(raw_datasets[dataset_name][text_data], raw_datasets[dataset_name]["label"], np.arange(len(raw_datasets[dataset_name])), test_size=0.25, random_state=42)

    preprocessed_datasets_dict[dataset_name, text_data] = DatasetDict()
    preprocessed_datasets_dict[dataset_name, text_data]["train"] = prepare_data(X_train, y_train)
    preprocessed_datasets_dict[dataset_name, text_data]["validation"] = prepare_data(X_test, y_test)

In [ ]:
for dataset_name in list(raw_datasets.keys()):
  for text_data in ["Title", "Abstract"]:
    print(dataset_name, text_data)
    print(preprocessed_datasets_dict[dataset_name, text_data])
    print()

LPI Title
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 4224
    })
    validation: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 1409
    })
})

LPI Abstract
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 4224
    })
    validation: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 1409
    })
})

PREDICTS Title
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 4152
    })
    validation: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 1384
    })
})

PREDICTS Abstract
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 4152
    })
    validation: Dataset({
        features: ['text', 'label', 'idx'],
        num_rows: 1384
    })
})



In [ ]:
tokenized_datasets_dict = {}

In [ ]:
tokenizer_dict = {}

for model_name, model_checkpoint in zip(model_names, checkpoint_names):
  for dataset_name in list(raw_datasets.keys()):
    for text_data in ["Title", "Abstract"]:

      tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
      tokenizer_dict[model_name] = AutoTokenizer.from_pretrained(model_checkpoint)

      tokenized_datasets_dict[dataset_name, text_data, model_name] = preprocessed_datasets_dict[dataset_name, text_data].map(tokenize_function, batched=True)
      tokenized_datasets_dict[dataset_name, text_data, model_name] = tokenized_datasets_dict[dataset_name, text_data, model_name].remove_columns(["text", "idx"])
      tokenized_datasets_dict[dataset_name, text_data, model_name] = tokenized_datasets_dict[dataset_name, text_data, model_name].rename_column("label", "labels")
      tokenized_datasets_dict[dataset_name, text_data, model_name].set_format("torch")

Map:   0%|          | 0/4224 [00:00<?, ? examples/s]

Map:   0%|          | 0/1409 [00:00<?, ? examples/s]

Map:   0%|          | 0/4224 [00:00<?, ? examples/s]

Map:   0%|          | 0/1409 [00:00<?, ? examples/s]

Map:   0%|          | 0/4152 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1409 [00:00<?, ? examples/s]

Map:   0%|          | 0/4152 [00:00<?, ? examples/s]

Map:   0%|          | 0/1384 [00:00<?, ? examples/s]

Map:   0%|          | 0/4152 [00:00<?, ? examples/s]

Map:   0%|          | 0/1384 [00:00<?, ? examples/s]

In [ ]:
data_collator_dict = {}
for model_name, model_checkpoint in zip(model_names, checkpoint_names):
  tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
  data_collator_dict[model_name] = DataCollatorWithPadding(tokenizer=tokenizer)

# Classical ML Models

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

results_list = []

for dataset_name in list(raw_datasets.keys()):
  for text_data in ["Title", "Abstract"]:
    print(dataset_name, text_data)
    X_train, X_test, \
    y_train, y_test, \
    indices_train, indices_test \
            = train_test_split(raw_datasets[dataset_name][text_data], raw_datasets[dataset_name]["label"], np.arange(len(raw_datasets[dataset_name])), test_size=0.25, random_state=42)

    vectorizer = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), max_df = 0.85)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    lr_mod = LogisticRegression(penalty='l2', class_weight='balanced').fit(X_train, y_train)
    y_predict = lr_mod.predict(X_test)

    results = calculate_scores(y_test, y_predict)

    results_list.append([dataset_name, text_data] + results + ["Logistic Regression"])
        
df_results_lr = pd.DataFrame(results_list, columns=["Dataset", "Text", "Accuracy", "Precision", "Recall", "F1-Score", "AUC", "Model"])

LPI Title
LPI Abstract
PREDICTS Title
PREDICTS Abstract


In [ ]:
df_results_lr

,Dataset,Text,Accuracy,Precision,Recall,F1-Score,AUC,Model
0,LPI,Title,0.932576,0.706731,0.812155,0.755784,0.881240,Logistic Regression
1,LPI,Abstract,0.960256,0.801932,0.917127,0.855670,0.941870,Logistic Regression
2,PREDICTS,Title,0.961705,0.780220,0.916129,0.842730,0.941791,Logistic Regression
3,PREDICTS,Abstract,0.973266,0.843023,0.935484,0.886850,0.956757,Logistic Regression


# Large Language Models

## Training

In [ ]:
model_dict = {}

In [ ]:
if torch.cuda.is_available():  # Tell PyTorch to use the GPU. 
 device = torch.device("cuda") 
 print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
 print('We will use the GPU:', torch.cuda.get_device_name(0)) # If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
for model_name, model_checkpoint in zip(model_names[:], checkpoint_names[:]):
  for dataset_name in list(raw_datasets.keys())[:]:
    for text_data in ["Title", "Abstract"][:]:
      print(model_name, dataset_name, text_data)

      model_dict[dataset_name, text_data, model_name] = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

      training_args = TrainingArguments(
          output_dir = "{}-finetuned-topicmodelling-{}".format(model_name, dataset_name+text_data),
          learning_rate = 2e-5,
          per_device_train_batch_size = 16,
          per_device_eval_batch_size = 16,
          num_train_epochs = 3,
          weight_decay = 0.01,
          evaluation_strategy = "epoch",
          # push_to_hub=True,
          fp16 = True
      )

      trainer = Trainer(
          model = model_dict[dataset_name, text_data, model_name],
          args = training_args,
          train_dataset = tokenized_datasets_dict[dataset_name, text_data, model_name]["train"],
          eval_dataset = tokenized_datasets_dict[dataset_name, text_data, model_name]["validation"],
          tokenizer = tokenizer_dict[model_name],
          data_collator = data_collator_dict[model_name]
      )

      trainer.train()

      # trainer.push_to_hub()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss
1,No log,0.255400
2,0.152600,0.145882
3,0.152600,0.173978


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss
1,No log,0.094291
2,0.119500,0.105696
3,0.119500,0.098975


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

...

Epoch,Training Loss,Validation Loss
1,No log,0.277801
2,0.112600,0.104732
3,0.112600,0.109369


Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

Epoch,Training Loss,Validation Loss
1,No log,0.203241
2,0.126500,0.090285
3,0.126500,0.099672


Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

Epoch,Training Loss,Validation Loss
1,No log,0.054712
2,0.098000,0.056865
3,0.098000,0.070533


## Evaluation

In [ ]:
results_list = []

for model_name, model_checkpoint in zip(model_names[:], checkpoint_names[:]):
  for dataset_name in list(raw_datasets.keys())[:]:
    for text_data in ["Title", "Abstract"][:]:
      
      eval_dataloader = DataLoader(
            tokenized_datasets_dict[dataset_name, text_data, model_name]["validation"], batch_size=8, collate_fn=data_collator_dict[model_name]
      )


      logit_list = []
      prediction_list = []
      labels_list = []
      model_dict[dataset_name, text_data, model_name].eval()
      for batch in eval_dataloader:
          batch = {k: v.to(device) for k, v in batch.items()}
          with torch.no_grad():
              outputs = model_dict[dataset_name, text_data, model_name](**batch)

          labels = batch["labels"]
          logits = outputs.logits
          predictions = torch.argmax(logits, dim=-1)
          logit_list.append(logits.cpu().detach().numpy())
          prediction_list.append(predictions.cpu().detach().numpy())
          labels_list.append(labels.cpu().detach().numpy())
      
      y_true = np.array([item for sublist in labels_list for item in sublist])
      y_predict = np.array([item for sublist in prediction_list for item in sublist])

      results = calculate_scores(y_true, y_predict)

      results_list.append([dataset_name, text_data] + results + [model_name])
          
df_results = pd.DataFrame(results_list, columns=["Dataset", "Text", "Accuracy", "Precision", "Recall", "F1-Score", "AUC", "Model"])

In [ ]:
df_results

,Dataset,Text,Accuracy,Precision,Recall,F1-Score,AUC,Model
0,LPI,Title,0.960256,0.874251,0.806630,0.839080,0.894764,BERT
1,LPI,Abstract,0.976579,0.906593,0.911602,0.909091,0.948879,BERT
2,PREDICTS,Title,0.968208,0.849057,0.870968,0.859873,0.925720,BERT
3,PREDICTS,Abstract,0.979046,0.875000,0.948387,0.910217,0.965650,BERT
4,LPI,Title,0.958126,0.896104,0.762431,0.823881,0.874701,BioBERT
5,LPI,Abstract,0.975160,0.924419,0.878453,0.900850,0.933933,BioBERT
6,PREDICTS,Title,0.973988,0.940741,0.819355,0.875862,0.906423,BioBERT
7,PREDICTS,Abstract,0.979769,0.899371,0.922581,0.910828,0.954781,BioBERT


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
